### Instalação dos pacotes necessários

In [4]:
! pip install unidecode
! pip install optuna
!pip install scikit-optimize
!pip install -U scikit-optimize

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.9/235.9 kB 5.9 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 365.7/365.7 kB 21.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.9/212.9 kB 22.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 10.0 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.3/100.3 kB 10.1 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


### Import das bibliotecas

In [5]:
import pandas as pd
import io
from google.colab import files
import pandas as pd
import numpy as np
from google.colab import files
import io
import unidecode
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix
import sklearn.metrics as metrics
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import precision_recall_curve
import matplotlib.pyplot as plt
from random import randrange
from sklearn.metrics import f1_score, precision_score, recall_score
from sklearn.impute import SimpleImputer
from sklearn import set_config
from sklearn.preprocessing import FunctionTransformer, RobustScaler
from collections import Counter
import random
import warnings
warnings.filterwarnings('ignore')
from imblearn.over_sampling import RandomOverSampler
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, KBinsDiscretizer
from imblearn.pipeline import make_pipeline
from sklearn.model_selection import GridSearchCV
from skopt.space import Real, Integer, Categorical
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from sklearn.metrics import classification_report
from skopt import BayesSearchCV
from joblib import parallel_backend
from sklearn.metrics import make_scorer, precision_score

### Carga dos dados

In [6]:
# Carregar o arquivo
uploaded = files.upload()
# Especificar a codificação correta
df = pd.read_csv(io.BytesIO(uploaded['full_devices.csv']), encoding='ISO-8859-1')

Saving full_devices.csv to full_devices.csv


### Avaliação da base

In [3]:
df.head()

,date,device,failure,attribute1,attribute2,attribute3,attribute4,attribute5,attribute6,attribute7,attribute8,attribute9
0,2015-01-01,S1F01085,0,215630672,56,0,52,6,407438,0,0,7
1,2015-01-01,S1F0166B,0,61370680,0,3,0,6,403174,0,0,0
2,2015-01-01,S1F01E6Y,0,173295968,0,0,0,12,237394,0,0,0
3,2015-01-01,S1F01JE0,0,79694024,0,0,0,6,410186,0,0,0
4,2015-01-01,S1F01R2B,0,135970480,0,0,0,15,313173,0,0,3


In [ ]:
df.describe()

,failure,attribute1,attribute2,attribute3,attribute4,attribute5,attribute6,attribute7,attribute8,attribute9
count,124494.000000,1.244940e+05,124494.000000,124494.000000,124494.000000,124494.000000,124494.000000,124494.000000,124494.000000,124494.000000
mean,0.000851,1.223881e+08,159.484762,9.940455,1.741120,14.222669,260172.657726,0.292528,0.292528,12.451524
std,0.029167,7.045933e+07,2179.657730,185.747321,22.908507,15.943028,99151.078547,7.436924,7.436924,191.425623
min,0.000000,0.000000e+00,0.000000,0.000000,0.000000,1.000000,8.000000,0.000000,0.000000,0.000000
25%,0.000000,6.128476e+07,0.000000,0.000000,0.000000,8.000000,221452.000000,0.000000,0.000000,0.000000
50%,0.000000,1.227974e+08,0.000000,0.000000,0.000000,10.000000,249799.500000,0.000000,0.000000,0.000000
75%,0.000000,1.833096e+08,0.000000,0.000000,0.000000,12.000000,310266.000000,0.000000,0.000000,0.000000
max,1.000000,2.441405e+08,64968.000000,24929.000000,1666.000000,98.000000,689161.000000,832.000000,832.000000,18701.000000


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 124494 entries, 0 to 124493
Data columns (total 12 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   date        124494 non-null  object
 1   device      124494 non-null  object
 2   failure     124494 non-null  int64 
 3   attribute1  124494 non-null  int64 
 4   attribute2  124494 non-null  int64 
 5   attribute3  124494 non-null  int64 
 6   attribute4  124494 non-null  int64 
 7   attribute5  124494 non-null  int64 
 8   attribute6  124494 non-null  int64 
 9   attribute7  124494 non-null  int64 
 10  attribute8  124494 non-null  int64 
 11  attribute9  124494 non-null  int64 
dtypes: int64(10), object(2)
memory usage: 11.4+ MB


### Separação da base para validação

In [ ]:
val_size = round(df.shape[0] * 0.2)
X_train = df.drop(['failure', 'date', 'device'], axis=1).iloc[val_size:].copy()
y_train = df['failure'].iloc[val_size:].copy()

X_val = df.drop(['failure', 'date', 'device'], axis=1).iloc[:val_size].copy()
y_val = df['failure'].iloc[:val_size].copy()

### Modelagem

#### Decision Tree com Random Search

In [3]:
# Definição do pipeline com a normalização, oversampling e modelo de árvore de decisão
pipeline = make_pipeline(StandardScaler(),
                         SMOTE(random_state=42),
                         DecisionTreeClassifier(random_state=42, class_weight='balanced'))

# Definição dos parâmetros para teste
param_dist = {
    'decisiontreeclassifier__max_depth': [2, 4, 6, 8, 10],
    'decisiontreeclassifier__min_samples_split': [2, 4, 6, 8, 10],
    'decisiontreeclassifier__min_samples_leaf': [1, 2, 3, 4, 5],
    'smote__sampling_strategy': ['minority', 'not majority', 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0],
    'decisiontreeclassifier__criterion': ['gini', 'entropy']
}

# Execução do RandomizedSearchCV
rand_search = RandomizedSearchCV(pipeline, param_distributions=param_dist,
                                 cv=5, scoring='f1', n_iter=20,
                                 random_state=42)
rand_search.fit(X_train, y_train)

# Treinamento com a melhor combinação de parâmetros
best_pipeline = rand_search.best_estimator_
best_pipeline.fit(X_train, y_train)

# Predição nos dados de treino
y_pred_train = best_pipeline.predict(X_train)
print("Train Classification Report:")
print(classification_report(y_train, y_pred_train))

# Predição nos dados de validação
y_pred_val = best_pipeline.predict(X_val)
print("Validation Classification Report:")
print(classification_report(y_val, y_pred_val))


Train Classification Report:
              precision    recall  f1-score   support

           0       1.00      0.94      0.97     99513
           1       0.01      0.56      0.02        82

    accuracy                           0.94     99595
   macro avg       0.50      0.75      0.49     99595
weighted avg       1.00      0.94      0.97     99595

Validation Classification Report:
              precision    recall  f1-score   support

           0       1.00      0.93      0.97     24875
           1       0.01      0.67      0.02        24

    accuracy                           0.93     24899
   macro avg       0.50      0.80      0.49     24899
weighted avg       1.00      0.93      0.96     24899



In [4]:
# Feature importance
tree_model = best_pipeline.named_steps['decisiontreeclassifier']
feat_importances = pd.Series(tree_model.feature_importances_, index=X_train.columns)
print("Feature Importance:")
print(feat_importances)

Feature Importance:
attribute1    0.065471
attribute2    0.122739
attribute3    0.010529
attribute4    0.457462
attribute5    0.093243
attribute6    0.146638
attribute7    0.041792
attribute8    0.036901
attribute9    0.025226
dtype: float64


In [3]:
# Definição do pipeline com a normalização, oversampling e modelo de árvore de decisão
pipeline = make_pipeline(StandardScaler(),
                         SMOTE(random_state=42),
                         DecisionTreeClassifier(random_state=42, class_weight={0: 1, 1: 5}, criterion='entropy'))

# Definição dos parâmetros para teste
param_dist = {
    'decisiontreeclassifier__max_depth': [2, 4, 6, 8, 10],
    'decisiontreeclassifier__min_samples_split': [2, 4, 6, 8, 10],
    'decisiontreeclassifier__min_samples_leaf': [1, 2, 3, 4, 5],
    'smote__sampling_strategy': ['minority', 'not majority', 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]
}

# Execução do RandomizedSearchCV
rand_search = RandomizedSearchCV(pipeline, param_distributions=param_dist,
                                 cv=5, scoring='f1', n_iter=20,
                                 random_state=42)
rand_search.fit(X_train, y_train)

# Treinamento com a melhor combinação de parâmetros
best_pipeline = rand_search.best_estimator_
best_pipeline.fit(X_train, y_train)

# Predição nos dados de treino
y_pred_train = best_pipeline.predict(X_train)
print("Train Classification Report:")
print(classification_report(y_train, y_pred_train))

# Predição nos dados de validação
y_pred_val = best_pipeline.predict(X_val)
print("Validation Classification Report:")
print(classification_report(y_val, y_pred_val))


Train Classification Report:
              precision    recall  f1-score   support

           0       1.00      0.93      0.96     99513
           1       0.01      0.90      0.02        82

    accuracy                           0.93     99595
   macro avg       0.51      0.92      0.49     99595
weighted avg       1.00      0.93      0.96     99595

Validation Classification Report:
              precision    recall  f1-score   support

           0       1.00      0.92      0.96     24875
           1       0.01      0.58      0.01        24

    accuracy                           0.92     24899
   macro avg       0.50      0.75      0.49     24899
weighted avg       1.00      0.92      0.96     24899



### Random Forest - Foco no equilibrio da identificação da falha

In [26]:
# Separa os dados em treino e teste
X_train_val, X_test, y_train_val, y_test = train_test_split(
    df.drop(['failure', 'date', 'device'], axis=1), df['failure'], test_size=0.2, random_state=42)

# Separa os dados de treino e validação
X_train, X_val, y_train, y_val = train_test_split(
    X_train_val, y_train_val, test_size=0.25, random_state=42)

# Define o modelo de classificação
model = RandomForestClassifier(n_estimators=100, random_state=42)

# Define a estratégia de oversampling
oversample = SMOTE(sampling_strategy=0.1, random_state=42)

# Aplica a técnica de oversampling no conjunto de treino
X_resampled, y_resampled = oversample.fit_resample(X_train, y_train)

# Treina o modelo com o conjunto de treino após resemplig usando validação cruzada
scores = cross_val_score(model, X_resampled, y_resampled, cv=5)

# Treina o modelo com o conjunto de treino após resemplig
model.fit(X_resampled, y_resampled)

# Realiza as predições no conjunto de teste
y_pred_test = model.predict(X_test)

# Calcula as métricas de avaliação para as previsões e rótulos verdadeiros do conjunto de teste
report_test = classification_report(y_test, y_pred_test)
print("Relatório de classificação para os dados de teste:\n", report_test)

# Realiza as predições no conjunto de validação
y_pred_val = model.predict(X_val)

# Calcula as métricas de avaliação para as previsões e rótulos verdadeiros do conjunto de validação
report_val = classification_report(y_val, y_pred_val)
print("Relatório de classificação para os dados de validação:\n", report_val)


Relatório de classificação para os dados de teste:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00     24881
           1       0.14      0.22      0.17        18

    accuracy                           1.00     24899
   macro avg       0.57      0.61      0.59     24899
weighted avg       1.00      1.00      1.00     24899

Relatório de classificação para os dados de validação:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00     24878
           1       0.08      0.10      0.09        21

    accuracy                           1.00     24899
   macro avg       0.54      0.55      0.54     24899
weighted avg       1.00      1.00      1.00     24899



## Random forest peso maior na manutenção

In [45]:
# Separa os dados em treino e teste
X_train_val, X_test, y_train_val, y_test = train_test_split(
    df.drop(['failure', 'date', 'device'], axis=1), df['failure'], test_size=0.2, random_state=42)

# Separa os dados de treino e validação
X_train, X_val, y_train, y_val = train_test_split(
    X_train_val, y_train_val, test_size=0.25, random_state=42)

# Define a estratégia de oversampling
oversample = SMOTE(sampling_strategy=0.1, random_state=42)

# Definição do espaço de busca para os hiperparâmetros
param_space = {
    'n_estimators': Integer(50, 100),
    'max_depth': Integer(5, 10),
    'min_samples_split': Integer(2, 5),
    'min_samples_leaf': Integer(1, 3),
    'bootstrap': Categorical([True, False])
}

# Define uma função de pontuação personalizada para calcular a precisão e recall da classe 1
def custom_scorer(y_true, y_pred):
    score = f1_score(y_true, y_pred, pos_label=1)
    return score

# Define o modelo de classificação com o parâmetro "class_weight"
model = RandomForestClassifier(random_state=42, class_weight={0: 1, 1: 5})

# Normaliza os dados de treino e validação
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)

# Converte a função custom_scorer em um scorer para ser usado pelo BayesSearchCV
scorer = make_scorer(custom_scorer)

# Realiza a busca por hiperparâmetros em paralelo
with parallel_backend('multiprocessing', n_jobs=-1):
    bayes_search = BayesSearchCV(
        estimator=model, search_spaces=param_space, cv=5, scoring=scorer, n_iter=5, n_jobs=-1
    )
    bayes_search.fit(X_train, y_train)

# Imprime os melhores hiperparâmetros encontrados
print("Melhores hiperparâmetros encontrados:")
print(bayes_search.best_params_)

# Treina o modelo com os melhores hiperparâmetros e o conjunto de treino após resemplig
best_model = bayes_search.best_estimator_
X_resampled, y_resampled = oversample.fit_resample(X_train, y_train)
best_model.fit(X_resampled, y_resampled)

# Normaliza os dados de teste
X_test = scaler.transform(X_test)

# Realiza as predições no conjunto de treino
y_pred_train = best_model.predict(X_resampled)

# Calcula as métricas de avaliação para as previsões e rótulos verdadeiros do conjunto de treino
report_train = classification_report(y_resampled, y_pred_train)
print("Relatório de classificação para os dados de treino:\n", report_train)

# Realiza as predições no conjunto de teste
y_pred_test = best_model.predict(X_test)

# Calcula as métricas de avaliação para as previsões e rótulos verdadeiros do conjunto de teste
report_test = classification_report(y_test, y_pred_test)
print("Relatório de classificação para os dados de teste:\n", report_test)

# Realiza as predições no conjunto de validação
y_pred_val = best_model.predict(X_val)

# Calcula as métricas de avaliação para as previsões e rótulos verdadeiros do conjunto de validação
report_val = classification_report(y_val, y_pred_val)
print("Relatório de classificação para os dados de validação:\n", report_val)

Melhores hiperparâmetros encontrados:
OrderedDict([('bootstrap', True), ('max_depth', 7), ('min_samples_leaf', 2), ('min_samples_split', 3), ('n_estimators', 100)])
Relatório de classificação para os dados de treino:
               precision    recall  f1-score   support

           0       0.99      0.88      0.93     74629
           1       0.89      0.99      0.94     74629

    accuracy                           0.94    149258
   macro avg       0.94      0.94      0.94    149258
weighted avg       0.94      0.94      0.94    149258

Relatório de classificação para os dados de teste:
               precision    recall  f1-score   support

           0       1.00      0.88      0.94     24881
           1       0.00      0.72      0.01        18

    accuracy                           0.88     24899
   macro avg       0.50      0.80      0.47     24899
weighted avg       1.00      0.88      0.93     24899

Relatório de classificação para os dados de validação:
               precis

In [34]:
# Separa os dados em treino e teste
X_train_val, X_test, y_train_val, y_test = train_test_split(
    df.drop(['failure', 'date', 'device'], axis=1), df['failure'], test_size=0.2, random_state=42)

# Separa os dados de treino e validação
X_train, X_val, y_train, y_val = train_test_split(
    X_train_val, y_train_val, test_size=0.25, random_state=42)

# Define a estratégia de oversampling
oversample = SMOTE(sampling_strategy=0.1, random_state=42)

# Definição do espaço de busca para os hiperparâmetros
param_space = {
    'n_estimators': Integer(50, 100),
    'max_depth': Integer(5, 10),
    'min_samples_split': Integer(2, 5),
    'min_samples_leaf': Integer(1, 3),
    'bootstrap': Categorical([True, False])
}

# Define uma função de pontuação personalizada para calcular a precisão e recall da classe 1
def custom_scorer(y_true, y_pred):
    score = f1_score(y_true, y_pred, pos_label=1)
    return score

# Define o modelo de classificação com o parâmetro "class_weight"
model = RandomForestClassifier(random_state=42, class_weight={0: 1, 1: 5})

# Normaliza os dados de treino e validação
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)

# Converte a função custom_scorer em um scorer para ser usado pelo BayesSearchCV
scorer = make_scorer(custom_scorer)

# Realiza a busca por hiperparâmetros em paralelo
with parallel_backend('multiprocessing', n_jobs=-1):
    bayes_search = BayesSearchCV(
        estimator=model, search_spaces=param_space, cv=5, scoring=scorer, n_iter=5, n_jobs=-1
    )
    bayes_search.fit(X_train, y_train)

# Imprime os melhores hiperparâmetros encontrados
print("Melhores hiperparâmetros encontrados:")
print(bayes_search.best_params_)

# Treina o modelo com os melhores hiperparâmetros e o conjunto de treino após resemplig
best_model = bayes_search.best_estimator_
X_resampled, y_resampled = oversample.fit_resample(X_train, y_train)
best_model.fit(X_resampled, y_resampled)

# Normaliza os dados de teste
X_test = scaler.transform(X_test)

# Realiza as predições no conjunto de treino
y_pred_train = np.argmax(best_model.predict_proba(X_resampled), axis=1)

# Calcula as métricas de avaliação para as previsões e rótulos verdadeiros do conjunto de treino
report_train = classification_report(y_resampled, y_pred_train)
print("Relatório de classificação para os dados de treino:\n", report_train)

# Realiza as predições no conjunto de teste
y_pred_test = np.argmax(best_model.predict_proba(X_test), axis=1)

# Calcula as métricas de avaliação para as previsões e rótulos verdadeiros do conjunto de teste
report_test = classification_report(y_test, y_pred_test)
print("Relatório de classificação para os dados de teste:\n", report_test)

# Realiza as predições no conjunto de validação
y_pred_val = np.argmax(best_model.predict_proba(X_val), axis=1)

# Calcula as métricas de avaliação para as previsões e rótulos verdadeiros do conjunto de validação
report_val = classification_report(y_val, y_pred_val)
print("Relatório de classificação para os dados de validação:\n", report_val)


Relatório de classificação para os dados de treino:
               precision    recall  f1-score   support

           0       0.98      0.98      0.98     74629
           1       0.80      0.85      0.82      7462

    accuracy                           0.97     82091
   macro avg       0.89      0.91      0.90     82091
weighted avg       0.97      0.97      0.97     82091

Relatório de classificação para os dados de teste:
               precision    recall  f1-score   support

           0       1.00      0.98      0.99     24881
           1       0.02      0.50      0.03        18

    accuracy                           0.98     24899
   macro avg       0.51      0.74      0.51     24899
weighted avg       1.00      0.98      0.99     24899

Relatório de classificação para os dados de validação:
               precision    recall  f1-score   support

           0       1.00      0.98      0.99     24878
           1       0.01      0.29      0.02        21

    accuracy         

### Predição em produção para falha

In [ ]:
# # Leitura do arquivo em produção
# df = pd.read_csv(io.BytesIO(uploaded['devices_prod.csv']), encoding='ISO-8859-1')

# # Normaliza os dados do dataframe
# X_prod = scaler.transform(df_prod)

# # Realiza as predições no conjunto de produção
# proba_falha = best_model.predict_proba(X_prod)[:,1]

# # Adiciona a coluna de probabilidade de falha ao dataframe
# df_prod['proba_falha'] = proba_falha

### Análises/Oportunidades
1. Oportunidade rodar o algoritmo em máquinas super potentes com mais núcleos visando aplicar gridsearch ou randomsearch em busca da melhor combinação de parâmetros
2. Coletar mais dados com a classe de falhar positiva
3. Coletar mais features
4. O melhor resultado que conseguimos com as limitações de hardware e base de dados leva a dois caminhos um seria um valor de recall da classe positiva maior visando acertar mais falando sobre a probabilidade alta de falha, contudo iria errar mais dizendo em casos que não terá falha e dizer que terá, ou buscar equilibrar precision e recall.

### Metodologia
1. Apliquei estratégicas para lhe dar com a base desbalanceada com o SMOTE que cria exemplos sintéticos com base em interpolação encontrando os k vizinhos mais próximos.
 2. Testei alguns dos modelos mais comuns e performáticos para esse tipo de tarefa com o árvore de decisão, random forest e XGBoost.
 3. Normalizei os dados, pois haviam features com escala muito diferente das outras, o que poderia impactar o resultado
 4. Foquei na análise do F1-Score como métrica para desafios desse tipo
 5. Utilizei algoritmos de busca para busca a melhor combinação de hiperparâmetros(Apenas o GridSearch poderia garantir o ótimo global, os demais apenas sub ótimos) a medida do possivel dado a necessidade de alta computação.
 6. Criei funções para tentar equilibrar os valores entre precision e recall
 7. Utilizei pipeline para organizar o código e evitar vazamento de dados
